# Handwritten Digit Classfication using Convolutional Neural Network

In this example, we are going to use the MNIST dataset to train a multi-layer feed foward neural network. MNIST is a simple computer vision dataset of handwritten digits. It has 60,000 training examles and 10,000 test examples. "It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting." For more details, please checkout the website [MNIST](http://yann.lecun.com/exdb/mnist/)

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Train the network

* prepare training and validation samples

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

In [4]:
//Parameters
val batchSize = 2048
val maxEpochs = 20
val learningRate = 0.2
val learningRateDecay = 0.0002

//Network Parameters
val nClasses = 10  //MNIST total classes (0-9 digits)

In [5]:
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize)
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize)

* create the LeNet-5 model

In [6]:
// Create a LeNet model
val model = Sequential()
model.add(Reshape(Array(1, 28, 28)))
model.add(SpatialConvolution(1, 6, 5, 5).setName("conv1"))
model.add(Tanh())
model.add(SpatialMaxPooling(2, 2, 2, 2).setName("pool1"))
model.add(Tanh())
model.add(SpatialConvolution(6, 12, 5, 5).setName("conv2"))
model.add(SpatialMaxPooling(2, 2, 2, 2).setName("pool2"))
model.add(Reshape(Array(12 * 4 * 4)))
model.add(Linear(12 * 4 * 4, 100).setName("fc1"))
model.add(Tanh())
model.add(Linear(100, nClasses).setName("score"))
model.add(LogSoftMax())

Sequential[e8160a75]{
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> output]
  (1): Reshape[df46fd98](1x28x28)
  (2): SpatialConvolution[conv1](1 -> 6, 5 x 5, 1, 1, 0, 0)
  (3): Tanh[3902fc5b]
  (4): SpatialMaxPooling[pool1](2, 2, 2, 2, 0, 0)
  (5): Tanh[ee0cc375]
  (6): SpatialConvolution[conv2](6 -> 12, 5 x 5, 1, 1, 0, 0)
  (7): SpatialMaxPooling[pool2](2, 2, 2, 2, 0, 0)
  (8): Reshape[35b4f8df](192)
  (9): Linear[fc1](192 -> 100)
  (10): Tanh[df74b1]
  (11): Linear[score](100 -> 10)
  (12): LogSoftMax[35d95790]
}

* configure optimizer

In [7]:
val optimizer = Optimizer(model = model, dataset = trainSet, criterion = CrossEntropyCriterion[Float]())
optimizer.setValidation(trigger = Trigger.everyEpoch, dataset = validationSet, vMethods = Array(new Top1Accuracy))
optimizer.setOptimMethod(new SGD(learningRate = learningRate, learningRateDecay = learningRateDecay))
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@588f72cc

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "lenet-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to lenet-20170925-025447

* Train the network. Wait some time till it finished.. Voila! You've got a trained model

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

can't find locality partition for partition 0 Partition locations are (ArrayBuffer(172.168.2.109)) Candidate partition locations are
(0,List()).
Optimization Done.

## 2. Inspect the prediction results

In [14]:
val rddData = sc.parallelize(load(validationData, validationLabel), batchSize)
val transformer = BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToSample()
val evaluationSet = transformer(rddData)
        
val result = trainedModel.evaluate(evaluationSet, Array(new Top1Accuracy[Float]), Some(batchSize))

result.foreach(r => println(s"${r._2} is ${r._1}"))

Top1Accuracy is Accuracy(correct: 9760, count: 10000, accuracy: 0.976)


In [15]:
val predictions = trainedModel.predict(evaluationSet)
val preLables = predictions.take(8).map(_.toTensor.max(1)._2.valueAt(1)).mkString(",")
val lables = evaluationSet.take(8).map(_.label.valueAt(1)).mkString(",")
println(preLables)
println(lables)

8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0
8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0


## 4. Draw the performance curve

In [16]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [17]:
val loss = trainSummary.readScalar("Loss")
val top1 = valSummary.readScalar("Top1Accuracy")

val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq

Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

val top1XY = top1.map(_ ._1).zip(top1.map(_ ._2)).toSeq
Vegas(description = "The Top1Accuracy curve.", width = 700.0, height = 300.0).
  withXY(top1XY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, bin = Bin(base = 0.9), title = "Top1Accuracy").
  mark(Line).
  show

<iframe id="frame-vegas-1a3c71a0-4555-4424-912a-696f4c175dcf" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-1a3c71a0-4555-4424-912a-696f4c175dcf'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 2.3183212280273438
 },
 {
 "x" : 2,
 "y" : 2.284992218017578
 },
 {
 "x" : 3,
 "y" : 2.254340648651123
 },
 {
 "x" : 4,
 "y" : 2.221071243286133
 },
 {
 "x" : 5,
 "y" : 2.1856091022491455
 },
 {
 "x" : 6,
 "y" : 2.1383118629455566
 },
 {
 "x" : 7,
 "y" : 2.084697961807251
 },
 {
 "x" : 8,
 "y" : 2.016324758529663
 },
 {
 "x" : 9,
 "y" : 1.9331648349761963
 },
 {
 "x" : 10,
 "y" : 1.8429282903671265
 },
 {
 "x" : 11,
 "y" : 1.7309292554855347
 },
 {
 "x" : 12,
 "y" : 1.6318382024765015
 },
 {
 "x" : 13,
 "y" : 1.524170160293579
 },
 {
 "x" : 14,
 "y" : 1.418565273284912
 },
 {
 "x" : 15,
 "y" : 1.3176456689834595
 },
 {
 "x" : 16,
 "y" : 1.198175072669983
 },
 {
 "x" : 17,
 "y" : 1.1289631128311157
 },
 {
 "x" : 18,
 "y" : 1.0444978475570679
 },
 {
 "x" : 19,
 "y" : 0.9479100704193115
 },
 {
 "x" : 20,
 "y" : 0.9054423570632935
 },
 {
 "x" : 21,
 "y" : 0.8589674830436707
 },
 {
 "x" : 22,
 "y" : 0.8052601218223572
 },
 {
 "x" : 23,
 "y" : 0.7770786881446838
 },
 {
 "x" : 24,
 "y" : 0.7330722212791443
 },
 {
 "x" : 25,
 "y" : 0.7180866003036499
 },
 {
 "x" : 26,
 "y" : 0.670555830001831
 },
 {
 "x" : 27,
 "y" : 0.6629337072372437
 },
 {
 "x" : 28,
 "y" : 0.6216341257095337
 },
 {
 "x" : 29,
 "y" : 0.6086056232452393
 },
 {
 "x" : 30,
 "y" : 0.5994284152984619
 },
 {
 "x" : 31,
 "y" : 0.5642670392990112
 },
 {
 "x" : 32,
 "y" : 0.5594971179962158
 },
 {
 "x" : 33,
 "y" : 0.5562388896942139
 },
 {
 "x" : 34,
 "y" : 0.5126533508300781
 },
 {
 "x" : 35,
 "y" : 0.5421416759490967
 },
 {
 "x" : 36,
 "y" : 0.4860299825668335
 },
 {
 "x" : 37,
 "y" : 0.5016460418701172
 },
 {
 "x" : 38,
 "y" : 0.49995777010917664
 },
 {
 "x" : 39,
 "y" : 0.493548721075058
 },
 {
 "x" : 40,
 "y" : 0.47896984219551086
 },
 {
 "x" : 41,
 "y" : 0.45399612188339233
 },
 {
 "x" : 42,
 "y" : 0.46575188636779785
 },
 {
 "x" : 43,
 "y" : 0.46428924798965454
 },
 {
 "x" : 44,
 "y" : 0.43324151635169983
 },
 {
 "x" : 45,
 "y" : 0.48079320788383484
 },
 {
 "x" : 46,
 "y" : 0.4270023703575134
 },
 {
 "x" : 47,
 "y" : 0.4503319263458252
 },
 {
 "x" : 48,
 "y" : 0.4282028079032898
 },
 {
 "x" : 49,
 "y" : 0.42802202701568604
 },
 {
 "x" : 50,
 "y" : 0.44490838050842285
 },
 {
 "x" : 51,
 "y" : 0.4342122972011566
 },
 {
 "x" : 52,
 "y" : 0.4372292160987854
 },
 {
 "x" : 53,
 "y" : 0.39485883712768555
 },
 {
 "x" : 54,
 "y" : 0.3777952790260315
 },
 {
 "x" : 55,
 "y" : 0.4077211320400238
 },
 {
 "x" : 56,
 "y" : 0.38488274812698364
 },
 {
 "x" : 57,
 "y" : 0.35855692625045776
 },
 {
 "x" : 58,
 "y" : 0.37062644958496094
 },
 {
 "x" : 59,
 "y" : 0.37284889817237854
 },
 {
 "x" : 60,
 "y" : 0.380959153175354
 },
 {
 "x" : 61,
 "y" : 0.3578607738018036
 },
 {
 "x" : 62,
 "y" : 0.366467148065567
 },
 {
 "x" : 63,
 "y" : 0.3752403259277344
 },
 {
 "x" : 64,
 "y" : 0.36971110105514526
 },
 {
 "x" : 65,
 "y" : 0.35463836789131165
 },
 {
 "x" : 66,
 "y" : 0.3414458632469177
 },
 {
 "x" : 67,
 "y" : 0.37226742506027

<iframe id="frame-vegas-ef981659-1233-4632-a273-7fee9c59f813" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-ef981659-1233-4632-a273-7fee9c59f813'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "bin" : {
 "base" : 0.9
 },
 "title" : "Top1Accuracy"
 }
 },
 "description" : "The Top1Accuracy curve.",
 "data" : {
 "values" : [
 {
 "x" : 30,
 "y" : 0.855400025844574
 },
 {
 "x" : 60,
 "y" : 0.9018999934196472
 },
 {
 "x" : 90,
 "y" : 0.923799991607666
 },
 {
 "x" : 120,
 "y" : 0.9350000023841858
 },
 {
 "x" : 150,
 "y" : 0.9419000148773193
 },
 {
 "x" : 180,
 "y" : 0.9452000260353088
 },
 {
 "x" : 210,
 "y" : 0.9532999992370605
 },
 {
 "x" : 240,
 "y" : 0.9573000073432922
 },
 {
 "x" : 270,
 "y" : 0.9614999890327454
 },
 {
 "x" : 300,
 "y" : 0.963699996471405
 },
 {
 "x" : 330,
 "y" : 0.964900016784668
 },
 {
 "x" : 360,
 "y" : 0.9664999842643738
 },
 {
 "x" : 390,
 "y" : 0.968500018119812
 },
 {
 "x" : 420,
 "y" : 0.9710000157356262
 },
 {
 "x" : 450,
 "y" : 0.9713000059127808
 },
 {
 "x" : 480,
 "y" : 0.9722999930381775
 },
 {
 "x" : 510,
 "y" : 0.9735999703407288
 },
 {
 "x" : 540,
 "y" : 0.9742000102996826
 },
 {
 "x" : 570,
 "y" : 0.975600004196167
 },
 {
 "x" : 600,
 "y" : 0.9761000275611877
 }
 ]
 }
}
 }
 vg.embed("#vegas-ef981659-1233-4632-a273-7fee9c59f813", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

Finally, the Spark should be stopped.

In [18]:
sc.stop()